In [ ]:
# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats# Setting up the environment.
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# Load the data from JHU
JHU_master_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
JHU_master_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

In [ ]:
# Remove all columns that are not dates so we can apply the daily_DF function to only columns with values for confirmed or deaths
JHU_c_dates = JHU_master_confirmed.drop(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' ,'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Province_State'], axis=1)
JHU_d_dates = JHU_master_deaths.drop(['Population', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2' , 'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Province_State'], axis=1)

In [ ]:
def daily_DF(df):
    # Copy the data frame
    dfdaily=df.copy()

    # We are on the first iteration
    first=True
    # For every column
    for column in df:
       # If this is the first iteration
       if first:
         # Save the first column
         prev = column
         # Copy over the first column unchanged
         dfdaily[column]=df[column]
         # All subsequent iterations are not the first
         first=False
         # Skip the rest of the loop
         continue
       # We are not the first iteration, so keep going
       # Set the new column contents to the difference of the
       # previous column and the current column
       dfdaily[column] = df[column]-df[prev]
       # The current column is the previous column for the next iteration
       prev = column
        
    # Convert negatives into zero
    num = dfdaily._get_numeric_data()
    num[num < 0] = 0
    
    return(dfdaily)

In [ ]:
# Apply the daily_DF function. We get a dataframe with daily increase values
JHU_confirmed_daily = daily_DF(JHU_c_dates)
JHU_deaths_daily = daily_DF(JHU_d_dates)

In [ ]:
# Extract some columns from the master data we got from JHU. We want to bring those coumns back into our daily increase dataframe.
JHU_c_non_dates = JHU_master_confirmed[['UID', 'iso2', 'code3', 'Admin2' , 'Combined_Key', 'Province_State']].copy()
JHU_d_non_dates = JHU_master_deaths[['UID', 'iso2', 'code3', 'Admin2' ,'Combined_Key', 'Population', 'Province_State']].copy()

In [ ]:
# Concatenate the columns from the original master table with the daily increase values
JHU_confirmed_as_columns = pd.concat([JHU_c_non_dates, JHU_confirmed_daily], axis=1)
JHU_deaths_as_columns = pd.concat([JHU_d_non_dates, JHU_deaths_daily], axis=1)

In [ ]:
# Use the melt function to pivot the table (we want dates as columns)
JHU_confirmed_melt = JHU_confirmed_as_columns.melt(id_vars =['UID', 'iso2', 'code3', 'Admin2' , 'Combined_Key', 'Province_State'])
JHU_confirmed = JHU_confirmed_melt.rename(columns={"variable": "Date", "value": "Confirmed"})

JHU_deaths_melt = JHU_deaths_as_columns.melt(id_vars = ['UID', 'iso2', 'code3', 'Admin2' , 'Combined_Key', 'Population', 'Province_State'])
JHU_deaths = JHU_deaths_melt.rename(columns={"variable": "Date", "value": "Deaths"})

In [ ]:
# Rename 0 to Confirmed
JHU_confirmed = JHU_confirmed.rename(columns={0: 'Confirmed'})

# Rename 0 to deaths
JHU_deaths = JHU_deaths.rename(columns={0: 'Deaths'})

# Rename Index column to Date
JHU_confirmed = JHU_confirmed.rename(columns={'index': 'Date'})
JHU_deaths = JHU_deaths.rename(columns={'index': 'Date'})

# Rename Admin2 column to County
JHU_confirmed = JHU_confirmed.rename(columns={'Admin2': 'County'})
JHU_deaths = JHU_deaths.rename(columns={'Admin2': 'County'})

In [ ]:
# Add/concatenate the Deaths column into the Confirmed table. We only want one table/dataframe
JHU_confirmed_and_deaths = pd.concat([JHU_confirmed, JHU_deaths[['Population', 'Deaths']]], axis=1)

In [ ]:
# We convert the Date column into Date/Time so we can find the max/latest date. It is also needed for the groupby operation later
import datetime as dt
JHU_confirmed_and_deaths['Date'] = JHU_confirmed_and_deaths['Date'].apply(lambda x:dt.datetime.strptime(x,'%m/%d/%y'))

In [ ]:
JHU_confirmed_and_deaths.dtypes

In [ ]:
# Rename confirmed_x and county_x to confirmed and county
JHU_confirmed_and_deaths = JHU_confirmed_and_deaths.rename(columns={"County_x": "County", "Confirmed_x": "Confirmed"})

In [ ]:
# We need to convert the UID to string (FIPS) to use in the map
JHU_confirmed_and_deaths['UID'] = JHU_confirmed_and_deaths['UID'].astype(str)

In [ ]:
# Cleaning the UID. Removing the number 840 (which is the US). The geojson from Plotly does not have it
JHU_confirmed_and_deaths['UID'] = JHU_confirmed_and_deaths['UID'].apply(lambda x : x[3:] if x.startswith('840') else x)

## Computing 23 Day Active values

In [ ]:
# Calculate the 23 day moving average for confirmed and deaths for all counties in all states
JHU_confirmed_and_deaths['Confirmed_23d_average'] = JHU_confirmed_and_deaths.groupby('Combined_Key')['Confirmed'].transform(lambda x: x.rolling(23,1).mean())

JHU_confirmed_and_deaths['Deaths_23d_average'] = JHU_confirmed_and_deaths.groupby('Combined_Key')['Deaths'].transform(lambda x: x.rolling(23,1).mean())

In [ ]:
# Calculate Active 23 days
JHU_confirmed_and_deaths['Active_23_days'] = (JHU_confirmed_and_deaths['Confirmed_23d_average'])*23

In [ ]:
# Calculate Estimated Active Cases (Confirmed Active in the last 23 days * 5)
JHU_confirmed_and_deaths['Estimated_active'] = JHU_confirmed_and_deaths['Active_23_days']*5

In [ ]:
# Converting negative active cases to zero
JHU_confirmed_and_deaths['Active_23_days'] = JHU_confirmed_and_deaths['Active_23_days'].mask(JHU_confirmed_and_deaths['Active_23_days'] < 0, 0)

In [ ]:
# Calculate percentage of population that is Active
JHU_confirmed_and_deaths['Percent_of_pop_active'] = JHU_confirmed_and_deaths['Estimated_active']/JHU_confirmed_and_deaths['Population']

In [ ]:
# Calculate Active 23 days per 100k
JHU_confirmed_and_deaths['Active_23_per_100k'] = (JHU_confirmed_and_deaths['Estimated_active']*100000)/JHU_confirmed_and_deaths['Population']

In [ ]:
last_update = JHU_confirmed_and_deaths['Date'].max()

In [260]:
JHU_c_and_d_WA = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Province_State'] == 'Washington']
JHU_c_and_d_WA.shape

(12218, 16)

In [316]:
import plotly.graph_objects as go
from scipy import signal 

fig = go.Figure()

unique_WA_counties = JHU_c_and_d_WA.County.unique().tolist()

for county in unique_WA_counties:
    b = JHU_c_and_d_WA['County'] == county
    fig.add_trace(go.Scatter(x=JHU_c_and_d_WA['Date'][b], 
    y= signal.savgol_filter(JHU_c_and_d_WA['Active_23_per_100k'][b], 7, 3),
    mode='lines',
    name=county,
    ))


fig.update_layout(title = dict(text='Active COVID19 Cases in Washington in the last 23 days<br> Updated: ' + str(last_update.strftime('%Y-%m-%d')), 
                               font = dict(color='#616161', size=14)),
                  title_x=0.5,
                  paper_bgcolor='rgba(0,0,0,0)', 
                  plot_bgcolor='rgba(0,0,0,0)',
                  height=1000,
                  uniformtext_minsize=7, uniformtext_mode='hide',
                  hovermode="closest",
                  hoverlabel=dict(
                                bgcolor="white", 
                                font_size=16, 
                                font_family="Arial",
                                bordercolor='#b3b3b3'),
                  font=dict(size = 12),
                  dragmode=False, #disable panning so mobile apps can receive pinch to zoom events
                 )
                 
                     
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='#a6a6a6', color='#4d4d4d',
                fixedrange=True,
                tickmode = 'array',
                spikethickness=0.4,
                spikedash='solid',
                nticks=20,
                tickfont = dict(color ='#9e9e9e'),
                )
              
fig.update_yaxes(showline=True, linewidth=1, linecolor='#a6a6a6', color='#4d4d4d',
                 showgrid=True, gridwidth=1, gridcolor='#f2f2f2',
                 fixedrange=True,
                 tickfont = dict(size = 14),
                 tickformat="0.4r"
                )

fig.show(displayModeBar=False)

import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)

In [ ]:
JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Active_23_days'].max()

In [ ]:
token = 'pk.eyJ1IjoiYWRyaW1vcyIsImEiOiJja2JqdnZ4bWEwYXZ0MnFzNHBub2RhYzhnIn0.KP_fYF_kLsFbU2IcJJl2bA'

loc = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Combined_Key']
per_inf = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Percent_of_pop_active']
infected = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Estimated_active']
active = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Active_23_days']
pop = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Population']
daily = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Confirmed']

mytext = ['{:}'.format(loc)+'<br><br>Percent of population infected: '+'<b>{:,.2%}</b>'.format(pi)+'<br>Calculated active cases: '+'{:,.0f}'.format(ac)+'<br>Estimated infection rate: '+'{:,.0f}'.format(ei)+'<br>Population: '+'{:,.0f}'.format(po)
         for loc, pi, ac, ei, po, in zip(list(loc), list(per_inf), list(active), list(infected), list(pop)) 
         ] 

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import plotly.graph_objects as go

fig = go.Figure(go.Choroplethmapbox(geojson = counties, 
                                    locations = JHU_confirmed_and_deaths['UID'], 
                                    z = JHU_confirmed_and_deaths.loc[JHU_confirmed_and_deaths['Date'] == last_update]['Percent_of_pop_active'],
                                    colorscale=[(0, '#fffcf5'),   (0.0001, '#fffcf5'),
                                                     (0.01, '#ffeBb9'), (0.025, '#facd91'),
                                                     (0.05, '#f6ad77'),  (0.1, '#8a1d63'), (1, '#2f0f3d')],
                                    zmin=0, 
                                    zmax=0.15,
                                    colorbar = dict(title='Percentage <br>of population <br>infected<br>',
                                    tickformat = ',.0%'), 
                                    marker_line_width=0.1,
                                    hoverlabel = dict(font_size = 16,
                                                      bgcolor = "#2f0f3d",
                                                      ),
                                    text = mytext,
                                    hoverinfo = 'text',
                                    hovertemplate = '%{text} <extra></extra>',
                                    ))


fig.update_layout(mapbox_style="light", mapbox_accesstoken=token,
                  mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129},
                  title_font=dict(size=12),
                  title_text='<b>Estimated percentage of population infected with COVID19</b> | Data: JHU & NY DOHMH | Updated: '+ str(last_update.strftime('%Y-%d-%m')) + '<br>Calculated Active Cases is defined as the average of cases over a 23 day period.<br>Esimated Infection Rate is defined as 5 times the number of Calculated Active Cases.<br><i>In collaboration with: Jay Schneider & Demaris Schneider</i>')
fig.update_layout(margin={"r":0,"t":110,"l":0,"b":50})
fig.show()

import plotly.io as pio
pio.write_html(fig, file='Index.html', auto_open=True)